In [ ]:
import pandas as pd, time, re, numpy as np
pd.options.mode.copy_on_write = True
from IPython.display import HTML
from functions_shared import stop_word, unzip_zip, prep_str_col, work_csv, adr_tag
from constant_vars import ZIPNAME, FRAMEWORK
from config_path import PATH_MATCH, PATH_SOURCE, PATH_CLEAN, PATH_ORG, PATH_WORK
from api_process.matcher import matcher

print(f"### IMPORT datasets")
participation = pd.read_pickle(f"{PATH_CLEAN}participation_current.pkl") 
participation['participation_linked'] = participation['project_id']+"-"+participation['orderNumber']
# participation = pd.read_pickle(f"{PATH_CLEAN}participation_complete.pkl")
entities_info = pd.read_pickle(f"{PATH_CLEAN}entities_info_current2.pkl")
# # entities = pd.read_pickle(f"{PATH_WORK}entities_participation_current.pkl")
proj = pd.read_pickle(f"{PATH_CLEAN}projects_current.pkl")
nuts = pd.read_pickle("data_files/nuts_complet.pkl")

countries = pd.read_pickle(f"{PATH_CLEAN}country_current.pkl")
lien = pd.read_pickle(f"{PATH_CLEAN}lien.pkl")
perso = pd.read_pickle(f"{PATH_CLEAN}persons_current.pkl")

pp_app = unzip_zip(ZIPNAME, f"{PATH_SOURCE}{FRAMEWORK}/", 'proposals_applicants_departments.json', 'utf8')
pp_app = pd.DataFrame(pp_app)
pp_app = pp_app.rename(columns={'proposalNbr':'project_id', 'applicantPic':'pic','departmentApplicantName':'department'}).astype(str)
pp_app = pp_app.replace({'None': np.nan})
print(f"- size pp_app: {len(pp_app)}")

pp_part = unzip_zip(ZIPNAME, f"{PATH_SOURCE}{FRAMEWORK}/", 'projects_participants_departments.json', 'utf8')
pp_part = pd.DataFrame(pp_part)
pp_part = pp_part.rename(columns={'projectNbr':'project_id', 'participantPic':'pic','departmentParticipantName':'department'}).astype(str)
pp_part = pp_part.replace({'None': np.nan})
print(f"- size pp_part: {len(pp_part)}")


### IMPORT datasets
- size pp_app: 380615
- size pp_part: 97205


C:\Users\zfriant\AppData\Local\Temp\ipykernel_15860\2420430520.py:30: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  pp_part = pp_part.replace({'None': np.nan})


In [48]:
participation.columns

Index(['project_id', 'generalPic', 'orderNumber', 'participation_linked',
       'erc_role', 'cordis_is_sme', 'flag_entreprise',
       'cordis_type_entity_code', 'cordis_type_entity_name_fr',
       'cordis_type_entity_name_en', 'cordis_type_entity_acro',
       'participation_nuts', 'region_1_name', 'region_2_name',
       'regional_unit_name', 'country_code', 'country_code_mapping',
       'extra_joint_organization', 'role', 'participates_as',
       'calculated_fund', 'stage', 'beneficiary_subv', 'coordination_number',
       'with_coord', 'is_ejo'],
      dtype='object')

In [8]:
entities_all = pd.concat([keep,  struct_et], ignore_index=True, axis=0)
print(f"size entities_all: {len(entities_all)}")


size entities_all: 596738


In [11]:
########
print("## add PERSO")
var_perso=['tel_clean', 'domaine_email', 'contact', 'num_nat_struct', 'email']
perso=(perso
    .mask(perso == '')
    .groupby(['project_id', 'generalPic', 'stage'], as_index=False)[var_perso]
    .agg(lambda x: ';'.join( x.dropna().unique()))
    .drop_duplicates())

print(f"size entities_all before perso: {len(entities_all)}")
tmp=(entities_all.drop(columns='_merge')
    .merge(perso, how='left', on=['project_id','generalPic', 'stage'], indicator=True))
print(f"size entities_all after perso: {len(tmp)}")

tmp1=tmp[tmp._merge=='both']

var_perso.append('_merge')
var_perso.remove('contact')
tmp2=(tmp[tmp._merge=='left_only']
    .drop(columns=var_perso)
    .merge(perso.drop(columns=['stage'])
    .drop_duplicates(), how='inner', on=['project_id','generalPic', 'contact']))

if len(tmp2)>0:
    # tmp=pd.concat([tmp[tmp._merge=='left_only'], tmp1, tmp2], ignore_index=True)
    print(f"A verifier code si tmp2 n'est pas null: {len(tmp)}")
else:
    entities_all=pd.concat([tmp[tmp._merge=='left_only'], tmp1], ignore_index=True)
    print(f"size entities_all after perso clean: {len(tmp)}")

#############
#merge des nouveaux nns

entities_all=entities_all.mask(entities_all=='')
entities_all['num_nat_struct'] = entities_all['num_nat_struct'].map(lambda x: x.split(';') if isinstance(x, str) else [])

entities_all.loc[entities_all.rnsr_back.str.len()>0, 'method'] = 'orga'
entities_all.loc[(entities_all.method.isnull())&(entities_all.rnsr_merged.str.len()>0), 'method'] = 'corda'
entities_all.loc[(entities_all.method.isnull())&(entities_all.num_nat_struct.str.len()>0), 'method'] = 'openalex'
entities_all.loc[(entities_all.method=='corda')&(entities_all.num_nat_struct.str.len()>0), 'method'] = entities_all.method+';openalex'
entities_all.loc[entities_all.method.str.contains('openalex', na=False), 'resultat'] = 'a controler'


entities_all.loc[entities_all.rnsr_merged.isnull(), 'rnsr_merged'] = entities_all.loc[entities_all.rnsr_merged.isnull(),'rnsr_merged'].apply(lambda x: [])
entities_all.loc[(entities_all.method=='corda')|(entities_all.method=='openalex'), 'rnsr_merged'] = entities_all.apply(lambda x: list(set(x['rnsr_merged'] + x['num_nat_struct'])), axis=1)

########################################



## add PERSO
size entities_all before perso: 596738
size entities_all after perso: 596738
size entities_all after perso clean: 596738


In [12]:
########
print("## geoloc cleaning")
stop_word(entities_all, 'country_code', ['street'])

## geoloc cleaning


In [13]:
tmp=entities_all.loc[(entities_all.country_code=='FRA')&(~entities_all.postalCode.isnull()), ['postalCode']].drop_duplicates()
tmp['code_postal'] = tmp.postalCode.str.replace(r"\D*", '', regex=True).str.strip()
tmp['code_postal'] = tmp.code_postal.map(lambda x: np.nan if len(x)!=5. else x)
tmp

,postalCode,code_postal
0,31029,31029
1,33600,33600
2,13572,13572
3,38610,38610
4,75794,75794
...,...,...
113727,68170,68170
113747,16220,16220
113749,31750,31750
113782,92514,92514


In [14]:
entities_all = pd.concat([entities_all, tmp.drop(columns='postalCode')], axis=1)
HTML(entities_all.loc[(entities_all.country_code=='FRA')&(~entities_all.city.isnull()), ['city']].drop_duplicates().sort_values('city').to_html())




In [17]:
entities_all[['postalCode', 'code_postal']].drop_duplicates()
# entities_all.loc[entities_all.code_postal.isnull(), 'code_postal'] = entities_all.loc[entities_all.code_postal.isnull(), 'postalCode']

,postalCode,code_postal
0,31029,31029
1,33600,33600
2,13572,13572
3,38610,38610
4,75794,75794
...,...,...
590114,11848,11848
591487,N52NB,N52NB
591917,111141,111141
592066,560066,560066


In [18]:
tmp = entities_all[['country_code','street_2']]
tmp = adr_tag(tmp, ['street_2'])

In [19]:

entities_all = pd.concat([entities_all.drop(columns='street_2'), tmp.drop(columns='country_code')], axis=1)

entities_all.loc[entities_all.country_code.isin(['FRA','BEL','LUX']), 'city'] = entities_all.city.str.replace(r"\bst\b", 'saint', regex=True).str.strip()
entities_all.loc[entities_all.country_code.isin(['FRA','BEL','LUX']), 'city'] = entities_all.city.str.replace(r"\bste\b", 'sainte', regex=True).str.strip()

entities_all.loc[~entities_all.city.isnull(), 'city_tag'] = entities_all.loc[~entities_all.city.isnull()].city.str.replace(r"\s+", '-', regex=True)

entities_all.to_pickle(f'{PATH_MATCH}entities_all.pkl')

In [ ]:
work_csv(tmp.loc[(tmp.country_code!='FRA')&(~tmp.street_2_tag.isnull()), 'street_2_tag'].drop_duplicates().sort_values(), 'city')


In [5]:
entities_all.loc[(entities_all.country_code!='FRA')&(~entities_all.street_2_tag.isnull()),'street_2_tag'] 
# = entities_all['street_2_tag'].str.strip()
# tmp=entities_all.copy()
# tmp.loc[(tmp.country_code!='FRA')&(~tmp.street_2_tag.isnull()), 'street_2_tag'] = tmp.loc[(tmp.country_code!='FRA')&(~tmp.street_2_tag.isnull())]['street_2_tag'].str.split(' ').apply(lambda x: [w for w in x if len(w) > 2])


8655                                            mickiewicza
8656                                            mickiewicza
8657                                   gran corts catalanes
8658                                   gran corts catalanes
8659                                          spitalstrasse
                                ...                        
596733    ronda auguste louis lumiere nave tecnologico v...
596734                                             valikatu
596735      tecnologico universidad miguel hernandez quorum
596736                                               orient
596737                                  kesklinn vana louna
Name: street_2_tag, Length: 546214, dtype: object

In [6]:
entities_all.loc[(entities_all.country_code!='FRA')&(~entities_all.street_2_tag.isnull()), ['country_code', 'street','street_2_tag']]

# tmp['v']=tmp['street_2_tag'].explode()
# tmp=tmp.explode('street_2_tag')


,country_code,street,street_2_tag
8655,POL,mickiewicza 30,mickiewicza
8656,POL,mickiewicza 30,mickiewicza
8657,ESP,gran via de les corts catalanes 587,gran corts catalanes
8658,ESP,gran via de les corts catalanes 587,gran corts catalanes
8659,CHE,spitalstrasse 33,spitalstrasse
...,...,...,...
596733,ESP,ronda auguste y louis lumiere 23 nave 13 parqu...,ronda auguste louis lumiere nave tecnologico v...
596734,FIN,valikatu 12,valikatu
596735,ESP,pq tecnologico universidad miguel hernandez sn...,tecnologico universidad miguel hernandez quorum
596736,DNK,orient plads 1,orient


In [ ]:
tmp['nb']=tmp.groupby(['country_code','street_2_tag'], as_index=False)['street_2_tag'].transform('count')

In [ ]:
work_csv(tmp.sort_values('street_2_tag'), 'city')

In [ ]:
lvar=['project_id', 'generalPic', 'role', 
       'title_clean', 'gender', 'email', 'tel_clean', 'domaine_email',
       'orcid_id', 'birth_country_code', 'nationality_country_code',
       'host_country_code', 'sending_country_code', 'iso2', 'stage', 'contact',
       'country_code', 'shift', 'call_year', 'thema_code', 'destination_code',
       'entities_id', 'entities_name', 'id_secondaire', 'country_code_mapping']
pp = pd.concat([perso_part.drop_duplicates(), perso_app.drop_duplicates()], ignore_index=True)

mask=((pp.country_code=='FRA')|(pp.nationality_country_code=='FRA')|(pp.destination_code.isin(['COG', 'PF', 'STG', 'ADG', 'POC','SyG', 'PERA', 'SJI'])))&(~(pp.contact.isnull()&pp.orcid_id.isnull()))
pp=pp.loc[mask, lvar].sort_values(['country_code','orcid_id'], ascending=False).drop_duplicates()
pp['contact2']=pp.contact.str.replace('-', ' ')

In [ ]:
def persons_files_import(thema, PATH_PERSONS):
    import re, os
    fname=''.join([filename for filename in os.listdir(PATH_PERSONS) if thema in filename])
    print(fname)

    if fname:
        with open(f"{PATH_PERSONS}{fname}", 'rb') as f:
            return pickle.load(f)

    if fname == []:
        fmax=max(int(os.path.splitext(filename)[0].split('_')[-1]) for filename in os.listdir(PATH_PERSONS) if re.search(r"persons_authors_[0-9]+",filename))
        if fmax:
            with open(f"{PATH_PERSONS}persons_authors_{fmax}.pkl", 'rb') as f:
                return pickle.load(f)

In [ ]:
with open(f'{PATH_PERSONS}persons_authors_erc_{CSV_DATE}.pkl', 'rb') as f:
    pers_ers = pickle.load(f)

In [ ]:
with open(f'{PATH_PERSONS}persons_authors_{CSV_DATE}.pkl', 'rb') as f:
    pers_api = pickle.load(f)

In [ ]:
def persons_api_simplify(df):
    pers = [] 
    for p in df:
        # elem = {k: v for k, v in p.items() if (v and v != "NaT")}

        p['institutions'] = []
        if p.get("affiliations"):
            for aff in p["affiliations"]:  
                res={"institution_name":aff.get('institution').get("display_name"),
                "institution_ror":aff.get('institution').get("ror"),
                "institution_country2":aff.get('institution').get("country_code"),
                "years":aff.get("years")}
                p['institutions'].append(res)
    
        p["orcid_openalex"] = p["ids"].get("orcid")            

        delete=['display_name_alternatives', 'topics', 'affiliations', 'id', 'last_known_institutions', 'ids']
        for field in delete:
            if p.get(field):
                p.pop(field)

        # elem = {k: v for k, v in elem.items() if (v and v != "NaT")}
        pers.append(p)

    print(len(pers))
    return pers

pers = persons_api_simplify(pers_api)
perc = persons_api_simplify(pers_ers)

In [ ]:
perc

In [ ]:
def persons_results_clean(df):
    from unidecode import unidecode
    from functions_shared import my_country_code, prop_string

    # df=pd.json_normalize(df, max_level=1)
    df=pd.json_normalize(df, record_path=['institutions'], meta=['match', 'orcid', 'display_name', 'orcid_openalex'], errors='ignore')
    df=df[~df.astype(str).duplicated()]
    cols = ['display_name']
    df = prop_string(df, cols)

    df['rows_by_name_orcid'] = df.groupby(['display_name', 'orcid'], dropna=False).transform('size')

    # persName_withOrcid_noAff=df[(df.match=='full_name')&(~df.orcid.isnull())&(df.institution_name.isnull())]
    # print(f"size person detect by name with an orcid but no affiliations: {len(persName_withOrcid_noAff)}")


    for i in ['orcid_openalex', 'orcid', 'institution_ror']:
        df.loc[~df[i].isnull(), i] = df.loc[~df[i].isnull()][i].str.split("/").str[-1]
    df['institution_ror'] = 'R'+ df['institution_ror'].astype(str)

    df['years']=df['years'].map(lambda liste: ';'.join(str(x) for x in liste))
    df=df[['match', 'display_name', 'orcid_openalex', 'years', 'institution_ror', 'institution_name', 'institution_country2', 'rows_by_name_orcid']]
    my_countries=my_country_code()
    df=(df.merge(my_countries[['iso2', 'iso3', 'parent_iso3']].drop_duplicates(), 
                 how='left', left_on='institution_country2', right_on='iso2')
        .drop(columns=['iso2'])
        .rename(columns={'iso3':'institution_country_map', 'parent_iso3':'institution_country'})
        )

    from step8_referentiels.paysage import paysage_prep
    from config_path import PATH
    DUMP_PATH=f'{PATH}referentiel/'
    paysage = paysage_prep(DUMP_PATH)
    df=(df.merge(paysage[['nom_long', 'numero_ror', 'numero_paysage', 'country_code_map', 'num_nat_struct']].drop_duplicates(), 
                 how='left', left_on='institution_ror', right_on='numero_ror'))



#     print(f"-3 size {match} cleaned: {len(df)}")
    return df

perc=persons_results_clean(perc)
pers=persons_results_clean(pers)


In [ ]:
temp=pd.concat([perc, pers], ignore_index=True)
temp=temp[~temp.astype(str).duplicated()]
pd.to_pickle(temp,f"{PATH_PERSONS}persons_{CSV_DATE}.pkl")
len(temp)

In [ ]:
pers.columns

In [ ]:
# temp=temp[~temp.astype(str).duplicated()]
pd.to_pickle(temp,f"{PATH_PERSONS}persons_{CSV_DATE}.pkl")
len(temp)

In [ ]:
# oth=pp.loc[~pp.thema_code.isin(['ERC', 'MSCA'])].drop_duplicates().reset_index(drop=True).sort_values('contact2')
# # pers_oth[pers_oth.display_name.str.contains('aubert')].display_name.unique()
# df=oth.merge(temp, how='inner', left_on=['contact2', 'country_code'], right_on=['display_name','iso3'])
# df=df[~df.astype(str).duplicated()]
# # df['years']=df['years'].map(lambda liste: ';'.join(str(x) for x in liste))
# df['filt']=df.apply(lambda x: x['call_year'] in x['years'], axis=1)

In [ ]:
# pers_oth[pers_oth.display_name.str.contains('aubert')].display_name.unique()
df=oth.merge(temp, how='inner', left_on=['contact2', 'country_code'], right_on=['display_name','iso3'])
df=df[~df.astype(str).duplicated()]
# df['years']=df['years'].map(lambda liste: ';'.join(str(x) for x in liste))
df['filt']=df.apply(lambda x: x['call_year'] in x['years'], axis=1)

df=df[['project_id', 'generalPic', 'role', 'orcid_id', 'orcid_openalex',
       'nationality_country_code', 'contact2',
        'stage', 'contact', 'display_name','country_code', 
        'entities_id', 'entities_name',
       'id_secondaire', 'country_code_mapping', 
        'institution_ror', 'institution_name',
       'institution_country',  'iso3', 'parent_iso3',
       'nom_long', 'numero_ror', 'numero_paysage', 'country_code_map',
       'num_nat_struct','rows_by_name_orcid','call_year','years', 'filt','thema_code', 'destination_code']]
# 

#extract entities_id begun by PIC
work_csv(df.loc[df.entities_id.str.startswith('pic', na=False), 
                ['generalPic', 'entities_name','id_secondaire', 'institution_ror', 'institution_name',
                'institution_country','numero_ror', 'numero_paysage', 'num_nat_struct']].drop_duplicates(), 'verif_id_struct_from openalex')

# #keep only rows with th same year
df1=df.loc[(df.filt==True)]


# df=df.loc[(df.filt==True)&(~df.num_nat_struct.isnull())]



In [ ]:
work_csv(df.loc[(df.filt==True)&(~df.num_nat_struct.isnull())], 'pers_with_nns')


In [ ]:
pp = pd.concat([perso_part.drop_duplicates(), perso_app.drop_duplicates()], ignore_index=True)
# mask=((pp.country_code=='FRA')|(pp.nationality_country_code=='FRA')|(pp.destination_code.isin(['COG', 'PF', 'STG', 'ADG', 'POC','SyG', 'PERA', 'SJI'])))&(~(pp.contact.isnull()&pp.orcid_id.isnull()))
pp=pp.sort_values(['country_code','orcid_id'], ascending=False).drop_duplicates()
pp['contact2']=pp.contact.str.replace('-', ' ')
len(pp)

In [ ]:
pp.loc[pp.contact=='zoubir khatir', ['stage','project_id', 'generalPic', 'role', 
       'title_clean', 'gender', 'email', 'tel_clean',
       'orcid_id', 'nationality_country_code',
       'host_country_code', 'contact',
       'country_code', 'call_year', 'thema_code', 'destination_code',
       'entities_id', 'entities_name', 'id_secondaire', 'country_code_mapping'
       ]]

In [ ]:
temp.loc[temp.display_name=='zoubir khatir']

In [ ]:
df=pp.merge(temp, how='inner', left_on=['contact2', 'country_code'], right_on=['display_name','iso3'])
df['filt_year']=df.apply(lambda x: x['call_year'] in x['years'], axis=1)
# same year
df=df.loc[(df.filt_year==True)]
#same entities
df1=df.loc[(df.entities_id==df.numero_paysage)]
df1=df1.assign(orcid_clean=np.where(df1.orcid_openalex.isnull(), df1.orcid_id, df1.orcid_openalex)).drop(columns='orcid_openalex').drop_duplicates()
# df1

In [ ]:
work_csv(df1, 'verif_orcid')

In [ ]:
# import time, pickle, requests
# from step7_persons.affiliations import openalex_name, openalex_orcid
# from config_path import PATH_CLEAN, PATH_API, PATH_WORK
# print(time.strftime("%H:%M:%S"))
# rlist=[]

# try:


#     author = {
#     "name": "caye pierre'",
#     "orcid": ""
#     }

#     if author.get("orcid"):
#         result = openalex_orcid(author)
#         if result.get('match'):
#             rlist.append(result)
#         else:
#             result = openalex_name(author)
#             if result:
#                 rlist.extend(result)
#     if author.get("orcid")=='':
#         result = openalex_name(author)
#         if result:
#             rlist.extend(result)

#     nf=f"persons_author"
#     with open(f'{PATH_WORK}test.pkl', 'wb') as f:
#         pickle.dump(rlist, f)
#     print(time.strftime("%H:%M:%S"))

# except requests.exceptions.HTTPError as http_err:
#     print(f"\n{time.strftime("%H:%M:%S")}, {author}-> HTTP error occurred: {http_err}")
#     nf=f"persons_author"
#     with open(f'{PATH_WORK}test.pkl', 'wb') as f:
#         pickle.dump(rlist, f)
#     print(time.strftime("%H:%M:%S"))

In [ ]:

with open(f'{PATH_WORK}test.pkl', 'rb') as f:
    test = pickle.load(f)
test

## return results from apenalex

1 - à partir d'orcid
2 - à partir du nom car orcid pas dans openalex -> vérifier que même personne ?
3 - à partir du nom car orcid non renseigné


In [ ]:
pers_api.mask(pers_api=='', inplace=True)
tmp1=pers_api.merge(my_countries, how='left', left_on='country_code', right_on='iso2')
# pers_api=country_clean(pers_api, ['country_code'])
tmp1=pers_api.loc[pers_api.match=='orcid']
len(tmp1)


In [ ]:
print(f"{len(perso_part)} ; {perso_part.columns}")

In [ ]:
tmp_part=perso_part.merge(tmp1, how='inner', left_on=['orcid_id'], right_on=['orcid'], suffixes=('','_api'))
tmp_part.loc[tmp_part.country_code!=tmp_part.country_code_api]

In [ ]:
data = [
    {
        "state": "Florida",
        "shortname": "FL",
        "info": {"governor": "Rick Scott"},
        "counties": [
            {"name": "Dade", "population": 12345},
            {"name": "Broward", "population": 40000},
            {"name": "Palm Beach", "population": 60000},
        ],
    },
    {
        "state": "Ohio",
        "shortname": "OH",
        "info": {"governor": "John Kasich"},
        "counties": [
            {"name": "Summit", "population": 1234},
            {"name": "Cuyahoga", "population": 1337},
        ],
    },
]
data

In [ ]:
def openalex_name(author):
    url = f"https://api.openalex.org/authors?filter=display_name.search:{author.get('name')}"
    nb_openalex=requests.get(url).json().get("meta").get('count')
    result={}
    if nb_openalex>0:
        for n in range(nb_openalex): 
            author_openalex = requests.get(url).json().get("results")[n]
            result.update({'display_name':author_openalex.get('display_name'), 'openalex_id':author_openalex.get('id'), 'affiliations':author_openalex.get('affiliations'), 'topics':author_openalex.get('topics'),  'x_concepts':author_openalex.get('x_concepts'), 'ids':author_openalex.get('ids'), 'display_name_alternatives':author_openalex.get('display_name_alternatives')})
            author.update(result)
            df=pd.concat([df, pd.json_normalize(author)])
    return df

In [ ]:
from config_path import PATH_API
import os, pandas as pd
 

for racine, repertoires, fichiers in os.walk(PATH_API):
    print(f"{racine}, {repertoires}, {fichiers}")
    for fichier in fichiers:
        if fichier.startswith('persons'):
            print(os.path.join(racine, fichier))
            globals()[f"{fichier}"]= pd.read_pickle(os.path.join(racine, fichier))


In [ ]:
fl=['persons_author.pkl', 'persons_author_10000.pkl', 'persons_author_12000.pkl', 'persons_author_2000.pkl', 'persons_author_4000.pkl', 'persons_author_6000.pkl', 'persons_author_8000.pkl', 'persons_author_name.pkl', 'persons_author_orcid.pkl']
for racine, repertoires, fichiers in os.walk(PATH_API):
    for i in fl:
        name=f"{i}".split('.')[0]
        print(name)
        globals()[name] = pd.read_pickle(os.path.join(racine, fichier))

In [ ]:
pers=pd.concat([persons_author_10000, persons_author_12000, persons_author_2000, persons_author_4000, persons_author_6000, persons_author_8000], ignore_index=True)

fl=['persons_author.pkl', 'persons_author_name.pkl', 'persons_author_orcid.pkl']

In [ ]:
def openalex_orcid(author):
    # from config_api import openalex_usermail
    import requests, time
    try:
        url = f"https://api.openalex.org/authors/orcid:{author.get('orcid')}?mailto=zmenesr@gmail.com"
        author_openalex = requests.get(url).json()
        result = author | {'display_name':author_openalex.get('display_name'), 
                           'openalex_id':author_openalex.get('id'), 
                           'affiliations':author_openalex.get('affiliations'), 
                           'topics':author_openalex.get('topics'),  
                           'x_concepts':author_openalex.get('x_concepts'), 
                           'ids':author_openalex.get('ids'), 
                           'display_name_alternatives':author_openalex.get('display_name_alternatives'),
                           'match':'orcid'}
        return result
    
    except requests.exceptions.HTTPError as http_err:
        print(f"\n{time.strftime("%H:%M:%S")}, {author}-> HTTP error occurred: {http_err}")
        return author
    except requests.exceptions.RequestException as err:
        print(f"\n{time.strftime("%H:%M:%S")}, {author}-> Error occurred: {err}")
        return author           
    except Exception as e:
        print(f"\n{time.strftime("%H:%M:%S")}, {author}-> An unexpected error occurred: {e}")
        return author

In [ ]:
def openalex_name(author):
    import time
    try:
        url = f"https://api.openalex.org/authors?filter=display_name.search:{author.get('name')}"
        nb_openalex=requests.get(url).json().get("meta").get('count')
        dl=[]
        if nb_openalex>0:
            print(nb_openalex)
            for n in range(nb_openalex): 
                author_openalex = requests.get(url).json().get("results")[n]
                if author_openalex.get('affiliations')!=[]:
                    result=author | {'display_name':author_openalex.get('display_name'), 
                                    'openalex_id':author_openalex.get('id'), 
                                    'affiliations':author_openalex.get('affiliations'),
                                    'topics':author_openalex.get('topics'),
                                    'x_concepts':author_openalex.get('x_concepts'), 
                                    'ids':author_openalex.get('ids'), 
                                    'display_name_alternatives':author_openalex.get('display_name_alternatives'),
                                    'match':'name'}
                    dl.append(result)
        return dl
    
    except requests.exceptions.HTTPError as http_err:
        print(f"\n{time.strftime("%H:%M:%S")}, {author}-> HTTP error occurred: {http_err}")
        return author
    except requests.exceptions.RequestException as err:
        print(f"\n{time.strftime("%H:%M:%S")}, {author}-> Error occurred: {err}")
        return author           
    except Exception as e:
        print(f"\n{time.strftime("%H:%M:%S")}, {author}-> An unexpected error occurred: {e}")
        return author
    

In [ ]:
from config_path import PATH_CLEAN
# from functions_shared import chunkify
perso_part = pd.read_pickle(f"{PATH_CLEAN}persons_participants.pkl")
# pp = pd.concat([perso_part[['contact', 'orcid_id', 'country_code']].drop_duplicates(), perso_app[['contact', 'orcid_id', 'country_code']].drop_duplicates()], ignore_index=True)
pp = perso_part[['contact', 'orcid_id', 'country_code']].fillna('')
pp=pp.loc[(pp.country_code=='FRA')].sort_values('orcid_id', ascending=False)
pp
data_chunks=list(chunkify(pp, 10000))
for i in range(0, len(data_chunks)):
    print(f"Loop {i}, size data_chunks: {len(data_chunks)}")
    # print(type(data_chunks))
    df_temp = data_chunks[i]
    

In [ ]:
import pickle, pandas as pd
from config_path import PATH_API
pers_api=[]
for i in range(1,3):
    print(i)
    with open(f"{PATH_API}fr_persons_author_{i}.pkl", 'rb') as f:
        globals()[f"pers_api{i}"] = pickle.load(f)
    pers_api.extend(globals()[f"pers_api{i}"])


In [ ]:
pers_api=pd.json_normalize(pers_api1, record_path=['affiliations'], meta=['name', 'orcid', 'display_name', 'openalex_id',  'match',  ["ids", "orcid"]],
        errors='ignore')

pers_api.columns = pers_api.columns.str.replace(r"[.*_]+", '_', regex=True)

pers_api = (pers_api
            .rename(columns={
                    'institution_country_code':'country_code'})
            .drop(columns=['institution_type','institution_lineage']))

for i in ['ids_orcid', 'institution_ror']:
    pers_api.loc[~pers_api[i].isnull(), i] = pers_api.loc[~pers_api[i].isnull()][i].str.split("/").str[-1]

In [ ]:
from config_path import PATH_CLEAN
perso_part = pd.read_pickle(f"{PATH_CLEAN}persons_participants.pkl")
perso_app = pd.read_pickle(f"{PATH_CLEAN}persons_applicants.pkl")

In [ ]:
with open(f"{PATH_API}persons_author.pkl", 'rb') as f:
    author_orcid = pickle.load(f)

In [ ]:
author_orcid=pd.json_normalize(author_orcid, record_path=['affiliations'], meta=['name','orcid', 'display_name', 'ids', 'match'])
author_orcid